<a href="https://colab.research.google.com/github/Nithesh-b/Twitter_Sentiment/blob/master/NLTKexample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import numpy as np 
print('numpy: {}'.format(np.__version__)) # Printing the version of Numpy

import pandas as pd
print('pandas: {}'.format(pd.__version__)) # Printing the version of pandas

import re
import matplotlib.pyplot as plt
%matplotlib inline

numpy: 1.18.5
pandas: 1.0.5


In [174]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from termcolor import colored
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [175]:
# Data read - All Required data  are in datafolder


InputdataFolder = "/content/sample_data/Base_tweets_DataSetV3.xlsx"


In [176]:
tweets_df = pd.read_excel(InputdataFolder)
print(tweets_df.shape)

(30256, 10)


In [177]:
#% of missing values in each column
def find_missing_values_func(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [178]:
#Call the find_missing_values_func(df)
columnsWiseMissingValue = find_missing_values_func(tweets_df) 
print(columnsWiseMissingValue)

Selected dataframe has 10 columns.
There are 5 columns that have missing values.
              Missing Values  % of Total Values
Date Created           10378               34.3
Tweet_source            2230                7.4
Tweet                    101                0.3
Country                   76                0.3
Tweeted-By                33                0.1


### class column creation

In [179]:
tweets_df

,tweet_id,SourceDataBase,OS,Tweet-Class_category-Code,Tweet_source,Tweeted-By,retweet_count,Tweet,Date Created,Country
0,neu-GG-Tweet-11945,MySQL,Windows,99,NaN,P32013,0,Just landed. My ears hurt,2016-02-27 11:35:52,Canada
1,neu-GG-Tweet-11944,MySQL,Windows,99,NaN,P32012,0,ouch following the #ASOT400 in TweetDeck excee...,2016-02-27 11:35:52,Canada
2,neu-GG-Tweet-11943,MySQL,Windows,99,NaN,P32011,0,@Nessiebear i really wanna see it! but no one ...,2016-02-27 11:35:52,Canada
3,neu-GG-Tweet-11942,MySQL,Windows,99,NaN,P32010,0,"@TessAaaaah Ahh, this is how you reply!! I had...",2016-02-27 11:35:52,Canada
4,neu-GG-Tweet-11941,MySQL,Windows,99,NaN,P32009,0,@chenez awwww. and i didn't get to be the hero...,2016-02-27 11:35:52,Canada
...,...,...,...,...,...,...,...,...,...,...
30251,neg-MG-12263,SQLSERVER,Windows,3,NaN,PMG12172,0,is not the new @AmericanAir is more like the n...,2017-11-06 09:51:15,USA
30252,neg-MG-12264,SQLSERVER,Windows,3,NaN,PMG12173,0,you're killing me from the inside,2016-02-27 11:35:52,USA
30253,neg-MG-12265,SQLSERVER,Windows,3,NaN,PMG12174,0,just hung up on me again. Another waste of an ...,2016-02-27 11:35:52,USA
30254,neg-MG-12266,SQLSERVER,Windows,3,NaN,PMG12175,0,and when will one of these agents be available...,2017-11-06 09:49:00,USA


In [180]:
tweets_df.iloc[:,0]

0        neu-GG-Tweet-11945
1        neu-GG-Tweet-11944
2        neu-GG-Tweet-11943
3        neu-GG-Tweet-11942
4        neu-GG-Tweet-11941
                ...        
30251          neg-MG-12263
30252          neg-MG-12264
30253          neg-MG-12265
30254          neg-MG-12266
30255          neg-MG-12267
Name: tweet_id, Length: 30256, dtype: object

In [181]:
#tweets_df.iloc[:,0] = tweets_df.iloc[:,0][:-12]
#Not working. Have to rework.

In [182]:
tweets_df.iloc[:,0].tail

<bound method NDFrame.tail of 0        neu-GG-Tweet-11945
1        neu-GG-Tweet-11944
2        neu-GG-Tweet-11943
3        neu-GG-Tweet-11942
4        neu-GG-Tweet-11941
                ...        
30251          neg-MG-12263
30252          neg-MG-12264
30253          neg-MG-12265
30254          neg-MG-12266
30255          neg-MG-12267
Name: tweet_id, Length: 30256, dtype: object>

#### Drop NULL Tweet-Text  rows as we use tweet text for text processing

In [183]:
#tweets_df = tweets_df.dropna(subset=[7])
#tweets_df = tweets_df[pd.notnull(tweets_df['Tweet'])]
tweets_df.dropna(subset=['Tweet'], how='all', inplace=True)
#Not sure if it is dropping any values

In [184]:
tweets_df

,tweet_id,SourceDataBase,OS,Tweet-Class_category-Code,Tweet_source,Tweeted-By,retweet_count,Tweet,Date Created,Country
0,neu-GG-Tweet-11945,MySQL,Windows,99,NaN,P32013,0,Just landed. My ears hurt,2016-02-27 11:35:52,Canada
1,neu-GG-Tweet-11944,MySQL,Windows,99,NaN,P32012,0,ouch following the #ASOT400 in TweetDeck excee...,2016-02-27 11:35:52,Canada
2,neu-GG-Tweet-11943,MySQL,Windows,99,NaN,P32011,0,@Nessiebear i really wanna see it! but no one ...,2016-02-27 11:35:52,Canada
3,neu-GG-Tweet-11942,MySQL,Windows,99,NaN,P32010,0,"@TessAaaaah Ahh, this is how you reply!! I had...",2016-02-27 11:35:52,Canada
4,neu-GG-Tweet-11941,MySQL,Windows,99,NaN,P32009,0,@chenez awwww. and i didn't get to be the hero...,2016-02-27 11:35:52,Canada
...,...,...,...,...,...,...,...,...,...,...
30251,neg-MG-12263,SQLSERVER,Windows,3,NaN,PMG12172,0,is not the new @AmericanAir is more like the n...,2017-11-06 09:51:15,USA
30252,neg-MG-12264,SQLSERVER,Windows,3,NaN,PMG12173,0,you're killing me from the inside,2016-02-27 11:35:52,USA
30253,neg-MG-12265,SQLSERVER,Windows,3,NaN,PMG12174,0,just hung up on me again. Another waste of an ...,2016-02-27 11:35:52,USA
30254,neg-MG-12266,SQLSERVER,Windows,3,NaN,PMG12175,0,and when will one of these agents be available...,2017-11-06 09:49:00,USA


In [185]:
columnsWiseMissingValue = find_missing_values_func(tweets_df) 
print(columnsWiseMissingValue)

Selected dataframe has 10 columns.
There are 3 columns that have missing values.
              Missing Values  % of Total Values
Date Created           10300               34.2
Tweet_source            2149                7.1
Tweeted-By                33                0.1


#### Drop Duplicate rows and duplicate tweet_id

In [186]:
tweets_df.drop_duplicates(subset ="tweet_id", keep = False, inplace = True) 

In [187]:
tweets_df

,tweet_id,SourceDataBase,OS,Tweet-Class_category-Code,Tweet_source,Tweeted-By,retweet_count,Tweet,Date Created,Country
0,neu-GG-Tweet-11945,MySQL,Windows,99,NaN,P32013,0,Just landed. My ears hurt,2016-02-27 11:35:52,Canada
1,neu-GG-Tweet-11944,MySQL,Windows,99,NaN,P32012,0,ouch following the #ASOT400 in TweetDeck excee...,2016-02-27 11:35:52,Canada
2,neu-GG-Tweet-11943,MySQL,Windows,99,NaN,P32011,0,@Nessiebear i really wanna see it! but no one ...,2016-02-27 11:35:52,Canada
3,neu-GG-Tweet-11942,MySQL,Windows,99,NaN,P32010,0,"@TessAaaaah Ahh, this is how you reply!! I had...",2016-02-27 11:35:52,Canada
4,neu-GG-Tweet-11941,MySQL,Windows,99,NaN,P32009,0,@chenez awwww. and i didn't get to be the hero...,2016-02-27 11:35:52,Canada
...,...,...,...,...,...,...,...,...,...,...
30251,neg-MG-12263,SQLSERVER,Windows,3,NaN,PMG12172,0,is not the new @AmericanAir is more like the n...,2017-11-06 09:51:15,USA
30252,neg-MG-12264,SQLSERVER,Windows,3,NaN,PMG12173,0,you're killing me from the inside,2016-02-27 11:35:52,USA
30253,neg-MG-12265,SQLSERVER,Windows,3,NaN,PMG12174,0,just hung up on me again. Another waste of an ...,2016-02-27 11:35:52,USA
30254,neg-MG-12266,SQLSERVER,Windows,3,NaN,PMG12175,0,and when will one of these agents be available...,2017-11-06 09:49:00,USA


#### Dropping "Date Created" column from main dataframe as it has no use

In [188]:
tweets_df = tweets_df.drop(columns=['Date Created','OS','SourceDataBase'])

In [189]:
tweets_df

,tweet_id,Tweet-Class_category-Code,Tweet_source,Tweeted-By,retweet_count,Tweet,Country
0,neu-GG-Tweet-11945,99,NaN,P32013,0,Just landed. My ears hurt,Canada
1,neu-GG-Tweet-11944,99,NaN,P32012,0,ouch following the #ASOT400 in TweetDeck excee...,Canada
2,neu-GG-Tweet-11943,99,NaN,P32011,0,@Nessiebear i really wanna see it! but no one ...,Canada
3,neu-GG-Tweet-11942,99,NaN,P32010,0,"@TessAaaaah Ahh, this is how you reply!! I had...",Canada
4,neu-GG-Tweet-11941,99,NaN,P32009,0,@chenez awwww. and i didn't get to be the hero...,Canada
...,...,...,...,...,...,...,...
30251,neg-MG-12263,3,NaN,PMG12172,0,is not the new @AmericanAir is more like the n...,USA
30252,neg-MG-12264,3,NaN,PMG12173,0,you're killing me from the inside,USA
30253,neg-MG-12265,3,NaN,PMG12174,0,just hung up on me again. Another waste of an ...,USA
30254,neg-MG-12266,3,NaN,PMG12175,0,and when will one of these agents be available...,USA


#### clean Tweet-Source  and replace missing tweet-source as "OTHERS"

In [190]:
tweets_df["Tweet_source"].fillna("OTHERS", inplace = True)  

In [191]:
tweets_df

,tweet_id,Tweet-Class_category-Code,Tweet_source,Tweeted-By,retweet_count,Tweet,Country
0,neu-GG-Tweet-11945,99,OTHERS,P32013,0,Just landed. My ears hurt,Canada
1,neu-GG-Tweet-11944,99,OTHERS,P32012,0,ouch following the #ASOT400 in TweetDeck excee...,Canada
2,neu-GG-Tweet-11943,99,OTHERS,P32011,0,@Nessiebear i really wanna see it! but no one ...,Canada
3,neu-GG-Tweet-11942,99,OTHERS,P32010,0,"@TessAaaaah Ahh, this is how you reply!! I had...",Canada
4,neu-GG-Tweet-11941,99,OTHERS,P32009,0,@chenez awwww. and i didn't get to be the hero...,Canada
...,...,...,...,...,...,...,...
30251,neg-MG-12263,3,OTHERS,PMG12172,0,is not the new @AmericanAir is more like the n...,USA
30252,neg-MG-12264,3,OTHERS,PMG12173,0,you're killing me from the inside,USA
30253,neg-MG-12265,3,OTHERS,PMG12174,0,just hung up on me again. Another waste of an ...,USA
30254,neg-MG-12266,3,OTHERS,PMG12175,0,and when will one of these agents be available...,USA



#### Tweet-Source - distribution - pie chart
#### Class distribution - also chart
#### Convert class  to numeric as - {'neu' : 2, 'pos' : 0, 'neg' : 1}
#### Explore Negative tweet and its catergory ( column - Tweet-Class_category-Code)
#### Country wise positive , Negative and Neutral
#### 